In [ ]:
import fitz  # PyMuPDF

# Ouvrir le fichier PDF
pdf_path = "texts/10_1.pdf"
with fitz.open(pdf_path) as pdf_document:
    texte_complet = ""
    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]
        lines = page.get_text().split('\n')
        texte_complet += page.get_text()  # Récupère le texte de chaque page

print("Texte extrait :", texte_complet)

text = texte_complet

In [ ]:
# Diviser le texte en blocs correspondant à chaque décision
import re

# Suppression des lignes contenant le motif variable (chiffres + MONITEUR BELGE)
#text = re.sub(r'\d{6}\s*BELGISCH STAATSBLAD — \d{2}\.\d{2}\.\d{4} — MONITEUR BELGE', '', text)

# Suppression des lignes contenant le motif variable (chiffres + sauts de ligne + MONITEUR BELGE)
text = re.sub(r'\d{6}+\s*BELGISCH\s+STAATSBLAD\s*—\s*\d{2}\.\d{2}\.\d{4}\s*—\s*MONITEUR\s*BELGE', '', text)

partie_pertinante_text = re.findall(
    r'(PUBLICATIONS LEGALES ET AVIS DIVERS.*?)'
    r'(?=Aanvaarding onder voorrecht van boedelbeschrijving|$)',
    text, 
    flags=re.DOTALL
)

print(partie_pertinante_text[0])

In [ ]:
blocks = re.findall(r'(Par ordonnance du.*?)(?=Par ordonnance du|$)', partie_pertinante_text[0], flags=re.DOTALL)

for block in blocks:
    print(block)

In [ ]:
import pandas as pd

data = []
for block in blocks:
    block = block.strip()
    if not block:
        continue  # Ignorer les blocs vides
    print(block, "\n---")
    # Supprimer les sauts de ligne et les espaces multiples pour faciliter la correspondance
    block_clean = ' '.join(block.split())

    # Expression régulière pour la mention légale
    legal_mention = r'conformément à l’article 492/1 de l’ancien Code civil et/ou à l’article 499/7 de l’ancien Code civil\.'

    # Expression régulière corrigée pour la personne protégée
    protected_person_pattern = re.compile(
        legal_mention +
        r'(?P<name>[^,]+?),\s*(?:né|née)(?:\s+à(?:\s+(?P<birth_place>[^,]+?))?)?\s+le\s+(?P<birth_date>\d{1,2} \w+ \d{4}),\s*' +
        r'(?:domicilié|domiciliée) à\s*(?P<address>.*?)(?=\s*(?:et résidant|résidant|personne à protéger|personne protégée|a été placé|a été placée|$))',
        flags=re.IGNORECASE
    )

    protected_person_match = protected_person_pattern.search(block_clean)

    # Gérer le cas où l'adresse est inconnue
    if not protected_person_match:

        protected_person_pattern = re.compile(
            legal_mention+
            r'(?P<name>[^,]+?),\s*(?:né|née)(?:\s+à(?:\s+(?P<birth_place>[^,]+?))?)?\s+le\s+(?P<birth_date>\d{1,2} \w+ \d{4}),\s*adresse inconnue',
            flags=re.IGNORECASE)
        protected_person_match = protected_person_pattern.search(block_clean)
        if protected_person_match:
            protected_person_address = 'Adresse inconnue'
            protected_person_name = protected_person_match.group('name').strip()
            protected_person_dob = protected_person_match.group('birth_date').strip()
            protected_person_birth_place = protected_person_match.group('birth_place').strip() if protected_person_match.group('birth_place') else 'Non spécifié'


        else:
            print("Informations de la personne protégée non trouvées dans le bloc :")
            print(block, "\n---")
            continue
    else:
        protected_person_address = protected_person_match.group('address').strip()
        protected_person_name = protected_person_match.group('name').strip()
        protected_person_dob = protected_person_match.group('birth_date').strip()
        protected_person_birth_place = protected_person_match.group('birth_place').strip() if protected_person_match.group('birth_place') else 'Non spécifié'

    administrator_mention = r'régime de représentation\.'

    # Expression régulière corrigée pour l'administrateur
    administrator_pattern = re.compile(
        administrator_mention+
        r'(?P<name>[^,]+?),\s*(?:avocat[e]?)?,?\s*(?:dont le cabinet (?:est(?: établi| situé| sis)?|sis) à )?\s*(?P<address>.*?)(?=,?\s*a été désigné[e]? en qualité d’administrateur)',
        flags=re.IGNORECASE)
    administrator_match = administrator_pattern.search(block_clean)

    if administrator_match:
        administrator_name = administrator_match.group('name').strip()
        administrator_address = administrator_match.group('address').strip()
    else:
        print("Informations de l'administrateur non trouvées dans le bloc :")
        print(block, "\n---")
        continue

    # Ajouter les données extraites à la liste
    data.append({
        'Protected Person Name': protected_person_name,
        'Protected Person Date of Birth': protected_person_dob,
        'Protected Person Birth Place': protected_person_birth_place if protected_person_birth_place else 'Non spécifié',
        'Protected Person Address': protected_person_address,
        'Administrator Name': administrator_name,
        'Administrator Address': administrator_address,
    })

# Créer un DataFrame à partir des données
df = pd.DataFrame(data)




In [ ]:
# Convertir les données en DataFrame
# Afficher le DataFrame
print("\nDataFrame output:")
print(df.to_markdown(index=False))
